# How to use the WattElse APIs from a third-party application ?

This notebook illustrate how to use the WattElse API.

The APIs have been developed using FastAPI. A documentation is available here: https://10.132.6.55:1978/docs

## Background: API authentication and security

For security reasons, the access to the APIs is secured based on different methods:
- https protocol
- use of authentication token
- access restricted to some groups for some API operations

Before to connect a new third-party application, you must apply for credentials.
The administrators will provide you an application secret that will be used to generate the authentication token.
Keep it securely!

In this example, we will use the following credentials:




In [ ]:
import os

client_id = "opfab"
client_secret = os.getenv("OPFAB_CLIENT_SECRET")
print(client_secret)

## Test of the RAG API

In [ ]:
import json
import requests

WATTELSE_API_URL = "https://10.132.6.55:1978"
TEST_GROUP_NAME = "OPFAB"

In [ ]:
# Authentication
response = requests.post(
        WATTELSE_API_URL + "/token",
        data={
            "username": client_id,  # Send client_id as username
            "password": client_secret,  # Send client_secret as password
        },
        verify=False,
    )

if response.status_code == 200:
        token_data = response.json()
        token = token_data["access_token"]
else:
  raise Exception(f"Failed to obtain token: {response.status_code} {response.text}")

headers = {"Authorization": f"Bearer {token}"}


In [ ]:
# Check if the service is running
response = requests.get(WATTELSE_API_URL + "/health", verify=False)
print(response.json())

In [ ]:
# List current sessions (1 session = 1 WattElse group = 1 document collection)
response = requests.get(WATTELSE_API_URL + "/current-sessions", verify=False)
print(response.json())

In [ ]:
# Create a new session
response = requests.post(WATTELSE_API_URL + "/create-session/" + TEST_GROUP_NAME, verify=False, headers=headers)
print(response.json())

In [ ]:
# Check the session was created
response = requests.get(WATTELSE_API_URL + "/current-sessions", verify=False, headers=headers)
print(response.json())

In [ ]:
# List available docs for the session
response = requests.get(WATTELSE_API_URL + "/list-available-docs/" + TEST_GROUP_NAME, verify=False, headers=headers)
print(response.json())

In [ ]:
# Upload documents
file_paths = ["NMT - Accord télétravail.pdf",]
files = [("files", open(p, "rb")) for p in file_paths]

response = requests.post(
    WATTELSE_API_URL + "/upload-docs/" + TEST_GROUP_NAME,
	files=files, verify=False, headers=headers
)
print(response.json())

In [ ]:
# Check documents were added
response = requests.get(WATTELSE_API_URL + "/list-available-docs/" + TEST_GROUP_NAME, verify=False, headers=headers)
print(response.json())

In [ ]:
# Query RAG
query = "Est-ce que je peux télétravailler à l'étranger ?"
response = requests.get(
            url=WATTELSE_API_URL + "/query-rag",
            data=json.dumps(
                {
                    "group_id": TEST_GROUP_NAME,
                    "message": query,
                    "history": None,
                    "group_system_prompt": None,
                    "selected_files": None,
                    "stream": False,
                }
            ),
            verify=False,
            headers=headers
)
print(response.json())

In [ ]:
# Remove documents
response = requests.post(
            url=WATTELSE_API_URL + "/remove-docs/" + TEST_GROUP_NAME,
            data=json.dumps(file_paths), verify=False, headers=headers
        )
response.json()

In [ ]:
# Clean sessions
response = requests.post(WATTELSE_API_URL + "/clean_sessions/" + TEST_GROUP_NAME, verify=False, headers=headers)
response.json()